In [1]:
number_of_shots = 7

In [2]:
experiment_name = "2024.05.08-03_Mixtral_7B_instruct_API_7-few-shots_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
# top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [10]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx")

In [11]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [12]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [13]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

# system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are {number_of_shots} examples of expected classification given an input text:

# """+few_shots+"""

# Answer with the label only using the format:---LABEL---
# """


In [14]:
data_sel = data #.iloc[:5]

In [15]:
# Sposta il prompt dentro il ciclo

In [16]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]    
    
    few_shots = ""

    for train_data_id in few_shots_list:

        # print("##################################################")
        # print(train_data_id)

        train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

        # print(train_row)

        # print(train_row.text.values[0])
        # print(train_row.sdg.values[0])

        input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
        output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

        few_shots += input_text
        few_shots += output_text    
    
    system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

    The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

    The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

    If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

    The following are {number_of_shots} examples of expected classification given an input text:

    """+few_shots+"""

    Answer with the label only using the format:---LABEL---
    """    
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 11---
1 6099 0  ---SDG 0--- 

(The text is about entertainment and does not seem to relate to any specific SDG.)
2 6471 0  ---SDG 16---

The text discusses the suspension of a life sentence awarded to a convict in a terror case and the court's observation that there was no prima facie evidence to prove that the convict had visited Bangladesh to collect funds for terror activities. The topic relates to the promotion of peaceful and inclusive societies for sustainable development, providing access to justice for all and building effective, accountable and inclusive institutions at all levels.
3 1786 0  ---SDG 4--- (Goal 4: Quality Education) as the text is about recognition of Indian scientists in the field of astronomy and education. The text highlights the naming of minor planets after four Indian scientists, including a Malayali, by the International Astronomical Union (IAU). This can be linked to Sustainable Development Goal 4, which aims to "ensure inclusive and equ

101 5269 1  ---SDG 1---

This text is about ensuring basic needs are met, which aligns with SDG 1's goal of poverty reduction.
102 3166 1  ---SDG 1---

The text discusses the measurement and analysis of child poverty and deprivation, which aligns with the focus of SDG 1: No Poverty. The paper aims to improve international understanding of issues relating to children's rights and to help facilitate full implementation of the Convention on the Rights of the Child, which is directly related to poverty reduction and improving children's lives.
103 4534 1  ---SDG 1---
104 1733 1  ---SDG 8---
105 2694 1  ---SDG 1---
106 6684 1  ---SDG 1---

The text is about the factors that increase a household's probability of being poor, including unemployment of the head of household, lack of electricity, and belonging to an ethnic group. This suggests that the text is focused on poverty reduction, which aligns with SDG 1: No Poverty.
107 173 1  ---SDG 1---
108 6136 1  ---SDG 1---
109 2652 1  ---SDG 1---

156 5694 2  ---SDG 2---
157 4433 2  ---SDG 2---

The text discusses the projected price behavior of various agricultural commodities, specifically cotton, beef, poultry, and sheep meats. This focus on food production and pricing aligns with SDG 2, which aims to end hunger, achieve food security, improve nutrition, and promote sustainable agriculture.
158 654 2  ---SDG 2---
159 4178 2  ---SDG 2---
160 5812 2  ---SDG 1---

The text emphasizes the importance of addressing income poverty in reducing undernourishment, which aligns with SDG 1's goal to end poverty in all its forms everywhere.
161 5130 2  ---SDG 2---
162 4078 2  ---SDG 8---
163 646 2  Based on the text provided, the experience described seems to be an example of child food insecurity, even if the child does not explicitly understand it as such. The text highlights the role of food in children's choices and responsibilities, as well as the stress associated with food insecurity, which suggests that the child is experiencing fo

253 5380 4  ---SDG 4---
254 2608 4  ---SDG 4---
255 823 4  ---SDG 4---
256 1972 4  ---SDG 4--- (Goal 4 is about ensuring inclusive and equitable quality education and promoting lifelong learning opportunities for all. The text is discussing education policies and funding, so this seems like the most appropriate goal.)
257 3492 4  ---SDG 4---
258 218 4  ---SDG 4---
259 3447 4  ---SDG 4---
260 3865 4  ---SDG 4---
261 932 4  ---SDG 4---
262 6608 4  ---SDG 4---
263 1276 4  ---SDG 4---
264 2037 4  ---SDG 4---
265 5181 4  ---SDG 4---
266 2261 4  ---SDG 4---
267 6446 4  ---SDG 9---

The text discusses the need for collaboration between higher education institutions and investment from the national government to improve innovation infrastructure, particularly in the traditional industry and service sector. This aligns with SDG 9, which aims to build resilient infrastructure, promote inclusive and sustainable industrialization, and foster innovation.
268 2771 4  ---SDG 4---
269 991 4  ---SDG 4-

415 5412 6  ---SDG 11---

The text discusses the use of Nature-Based Solutions (NBS) in treating industrial wastewater, which can contribute to improving the quality of life in urban areas, making it align with SDG 11: Sustainable Cities and Communities.
416 1548 6  ---SDG 14---
417 616 6  ---SDG 7---
418 3942 6  ---SDG 6---
419 2267 6  ---SDG 6---
420 1228 7  ---SDG 7---
421 5922 7  ---SDG 7---
422 3438 7  ---SDG 7---
423 4989 7  ---SDG 7---
424 1802 7  ---SDG 9---
425 6417 7  ---SDG 7---
426 3138 7  ---SDG 7---
427 6251 7  ---SDG 13---
428 6074 7  ---SDG 13---
429 5346 7  ---SDG 7---
430 6226 7  ---SDG 13---

The text discusses the need for government intervention to internalize the external costs of greenhouse gas (GHG) emissions in production and consumption activities. This is closely related to climate action (SDG 13), as pricing emissions will encourage the exploitation of abatement opportunities and reduce GHG emissions.
431 3417 7  ---SDG 7---
432 3016 7  ---SDG 9---

The text

...

631 2869 10  ---SDG 8---
632 4938 10  ---SDG 10---
633 3918 10  ---SDG 10---
634 6658 10  ---SDG 10---
635 2063 10  ---SDG 1---

The text emphasizes the reduction of poverty and income inequality in Brazil, which is the main goal of SDG 1. The improvement is attributed to economic growth and effective social policies, including the "Bolsa Familia" program, which is a conditional cash transfer program. The text also highlights the importance of education in reducing poverty and income inequality, which is another target of SDG 1. Therefore, the text can be classified as---SDG 1---.
636 93 10  ---SDG 4---
637 1205 10  ---SDG 10---
638 6771 10  ---SDG 1---

The text discusses the impact of public services on poorer households, with efficiency gains potentially compensating for lower spending. The focus on improving the living conditions of the most vulnerable populations aligns with SDG 1, which aims to end poverty in all its forms everywhere.
639 4304 10  ---SDG 9---
640 2352 10  ---SDG 

705 2640 11  ---SDG 11---
706 2678 11  ---SDG 11---
707 1725 11  ---SDG 11---

The text is about road safety measures in the London area, which aligns with SDG 11's focus on making cities inclusive, safe, resilient, and sustainable. The emphasis on preventive measures and reducing the severity of injuries post-collision supports this classification.
708 2798 11  ---SDG 11---
709 172 11  ---SDG 11---

The text discusses the development and approval process of comprehensive development plans (CDPs) for small towns and monotowns in a particular region. The focus is on the planning and coordination efforts for the physical and economic development of these areas, which aligns with SDG 11: Sustainable Cities and Communities. However, the text does not mention any aspects related to the other SDGs, so the most appropriate classification is SDG 11.
710 2226 11  ---SDG 11---
711 1560 11  ---SDG 11---
712 3315 11  ---SDG 11---
713 5109 11  ---SDG 11---
714 5983 11  ---SDG 11---
715 4398 11  ---

771 1616 12  ---SDG 12---
772 5617 12  ---SDG 12---
773 2243 12  ---SDG 12---
774 1134 12  ---SDG 9---
775 1049 12  ---SDG 12---
776 430 12  ---SDG 12---
777 2354 12  ---SDG 12---
(This text is about waste management and recycling, which aligns with SDG 12: Responsible Consumption and Production)
778 141 12  ---SDG 12---
779 126 12  ---SDG 12---
780 3805 13  ---SDG 13---
781 2862 13  ---SDG 13---
782 2180 13  ---SDG 13---
783 2676 13  ---SDG 13---
784 5685 13  ---SDG 13---
785 2312 13  ---SDG 13---
786 1062 13  ---SDG 13---
787 2293 13  ---SDG 13---
788 760 13  ---SDG 13---
789 611 13  ---SDG 13---

The text discusses issues related to climate finance in the context of the UNFCCC and the Paris Agreement. The focus is on the transparency and reporting of climate finance, which falls under SDG 13 (Climate Action). The text does not contain information that would classify it under any other SDG.
790 3633 13  ---SDG 15---
791 4001 13  ---SDG 13---
792 2228 13  ---SDG 13---
793 2575 13  ---

874 3404 14  ---SDG 14---
875 2212 14  ---SDG 14---
876 4014 14  ---SDG 2---
877 1200 14  Based on the given text, the classification under the Sustainable Development Goals (SDGs) is:

---SDG 14---

Explanation:

The text discusses the potential of aquaculture, the growth of the sector, and the challenges it faces, including environmental issues and potential impacts on biodiversity and ecosystem services. The text also mentions the need for sustainable growth and the importance of addressing the sustainability of the sector. These themes are closely related to SDG 14, which aims to conserve and sustainably use the oceans, seas, and marine resources for sustainable development.
878 2525 14  ---SDG 14---
879 864 14  ---SDG 14---
880 3804 14  ---SDG 14---
881 440 14  ---SDG 14---
882 6698 14  ---SDG 14---
883 1348 14  ---SDG 14---
884 4396 14  ---SDG 14---
885 3059 14  ---SDG 14---
886 2693 14  ---SDG 14---
887 5734 14  ---SDG 14---
888 1089 14  ---SDG 2---
889 5290 14  ---SDG 14---
890

979 64 16  ---SDG 16---
980 3614 16  ---SDG 16---
981 4520 16  ---SDG 3---
982 569 16  ---SDG 16---
983 543 16  ---SDG 16---
984 4331 16  ---SDG 16---
985 5742 16  ---SDG 16---
986 1384 16  ---SDG 16---
987 2092 16  ---SDG 5---
988 2472 16  ---SDG 12---
989 3167 16  ---SDG 16---
990 5026 16  ---SDG 16---
991 2834 16  ---SDG 16---
992 2469 16  ---SDG 16---
993 1609 16  ---SDG 17---

The text discusses the potential of using a human resource management model from the U.S. Intelligence Community for civil service reform in the Obama administration. This topic relates to SDG 17, which is about strengthening the means of implementation and revitalizing the global partnership for sustainable development. Specifically, this goal aims to enhance policy coherence for sustainable development, promote effective public, public-private and civil society partnerships, and increase access to technology. The text highlights the importance of sharing best practices and lessons learned in human resource

In [17]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [18]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    #print(res)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
        sdg_pred = res[3]['choices'][0]['message']['content']
    except:
        sdg_pred = res
    res4df = (res[0], res[1], res[2], sdg_pred)    
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---')
('582', 714, 9, ' ---SDG 9---')
('2', 6471, 0, " ---SDG 16---\n\nThe text discusses the suspension of a life sentence awarded to a convict in a terror case and the court's observation that there was no prima facie evidence to prove that the convict had visited Bangladesh to collect funds for terror activities. The topic relates to the promotion of peaceful and inclusive societies for sustainable development, providing access to justice for all and building effective, accountable and inclusive institutions at all levels.")
('782', 2180, 13, ' ---SDG 13---')
('601', 4947, 10, ' ---SDG 10---')
('144', 2727, 2, ' ---SDG 2---')
('54', 1497, 0, ' ---SDG 9---')
('125', 5756, 2, " ---SDG 11---\n\nThe text is about rural development policies and how private and public infrastructure investments in rural areas can impact the local economy and livelihoods. The focus is on the importance of well-defined property rights, the role of social enterprises and communi

In [19]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [20]:
results_processed.head(3)

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---
1,582,714,9,---SDG 9---
2,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...


In [21]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\n(The text is about entertainm...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...


In [22]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\n(The text is about entertainm...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...,16


In [23]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,30,30,0,30,30,0,0,0,30,30,30,30,30
1,75,75,74,75,75,74,74,74,75,75,75,75,75
10,35,35,35,35,35,35,35,35,35,35,35,35,35
11,82,82,77,82,82,77,77,77,82,82,82,82,82
12,55,55,55,55,55,55,55,55,55,55,55,55,55
13,66,66,65,66,66,65,65,65,66,66,66,66,66
14,54,54,54,54,54,54,54,54,54,54,54,54,54
15,57,57,55,57,57,55,55,55,57,57,57,57,57
16,55,55,51,55,55,51,51,51,55,55,55,55,55


In [24]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [25]:
results_processed_merged = results_processed_merged[~results_processed_merged.predicted_label_clean.isna()]

In [26]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1018
1018
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        60
           1       0.72      0.90      0.80        60
           2       0.89      0.80      0.84        60
           3       0.81      0.95      0.88        60
           4       0.82      0.93      0.87        60
           5       0.88      0.88      0.88        60
           6       1.00      0.85      0.92        60
           7       0.91      0.82      0.86        60
           8       0.57      0.72      0.64        60
           9       0.66      0.78      0.72        60
          10       0.89      0.52      0.65        60
          11       0.71      0.97      0.82        60
          12       0.91      0.83      0.87        60
          13       0.79      0.88      0.83        59
          14       0.96      0.87      0.91        60
          15       0.93      0.88      0.91        60
          16       0.89      0.83      0.86        59
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# AGGIUNTO IL 2024.05.07:

In [28]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [29]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\n(The text is about entertainm...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...,16


In [30]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1018
1018
              precision    recall  f1-score   support

           0       0.83      0.50      0.62        60
           1       0.72      0.90      0.80        60
           2       0.89      0.80      0.84        60
           3       0.81      0.95      0.88        60
           4       0.82      0.93      0.87        60
           5       0.88      0.88      0.88        60
           6       1.00      0.85      0.92        60
           7       0.91      0.82      0.86        60
           8       0.57      0.72      0.64        60
           9       0.66      0.78      0.72        60
          10       0.89      0.52      0.65        60
          11       0.71      0.97      0.82        60
          12       0.91      0.83      0.87        60
          13       0.79      0.88      0.83        59
          14       0.96      0.87      0.91        60
          15       0.93      0.88      0.91        60
          16       0.89      0.83      0.86        59

    accuracy    

In [31]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [32]:
report_df

,precision,recall,f1-score,support
0,83.333333,50.000000,62.500000,60.000000
1,72.000000,90.000000,80.000000,60.000000
2,88.888889,80.000000,84.210526,60.000000
3,81.428571,95.000000,87.692308,60.000000
4,82.352941,93.333333,87.500000,60.000000
5,88.333333,88.333333,88.333333,60.000000
6,100.000000,85.000000,91.891892,60.000000
7,90.740741,81.666667,85.964912,60.000000
8,57.333333,71.666667,63.703704,60.000000
9,66.197183,78.333333,71.755725,60.000000


In [33]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [34]:
print(precision, recall, accuracy, f1_score)

83.4104760262524 81.8344965104686 81.82711198428291 81.67373493776095


In [35]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

83.4 81.8 81.8 81.7
